## Predicting the Stock Market

We'll be using historical data on the price of the S&P500 Index to make predictions about future prices. Predicting whether an index will go up or down will help us forecast how the stock market as a whole will perform. Since stocks tend to correlate with how well the economy as a whole is performing, it can also help us make economic forecasts.
We'll be using this dataset to develop a predictive model. We'll train the model with data from 1950-2012, and try to make predictions from 2013-2015.

In [1]:
import pandas as pd
from datetime import datetime

df = pd.read_csv('sphist.csv')
df['Date'] = pd.to_datetime(df['Date'])

df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883


In [2]:
df = df.sort_values(by=['Date'], ascending=True)
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


### Creating Indicators

Datasets taken from the stock market need to be handled differently than datasets from other sectors when it comes time to make predictions. This means we have to be extra careful to not inject "future" knowledge into past rows when we do training and prediction. Injecting future knowledge will make our model look good when we're training and testing it, but will make it fail in the real world. The time series nature of the data means that can generate indicators to make our model more accurate.

Here are some indicators that are interesting to generate for each row:

- The average price from the past 5 days.
- The average price for the past 30 days.
- The average price for the past 365 days.
- The ratio between the average price for the past 5 days, and the average price for the past 365 days.
- The standard deviation of the price over the past 5 days.
- The standard deviation of the price over the past 365 days.
- The ratio between the standard deviation for the past 5 days, and the standard deviation for the past 365 days.
- The average volume over the past five days.
- The average volume over the past year.
- The ratio between the average volume for the past five days, and the average volume for the past year.
- The standard deviation of the average volume over the past five days.
- The standard deviation of the average volume over the past year.
- The ratio between the standard deviation of the average volume for the past five days, and the standard deviation of the average volume for the past year.
- The year component of the date.
- The ratio between the lowest price in the past year and the current price.
- The ratio between the highest price in the past year and the current price.
- The year component of the date.
- The month component of the date.
- The day of week.
- The day component of the date.
- The number of holidays in the prior month.


In [3]:
# Average price from the past 5 days (present day exluded)
df['avg_close_5'] = df.Close.rolling(5).mean().shift(1, axis=0)

# Average price from the past 30 days (present day exluded)
df['avg_close_30'] = df.Close.rolling(30).mean().shift(1, axis=0)

# Standard deviation of the price over the past 365 days (present day exluded)
df['std_close_365'] = df.Close.rolling(365).std().shift(1, axis=0)

# Average volume from the past 30 days (present day exluded)
df['avg_vol_5'] = df.Volume.rolling(5).mean().shift(1, axis=0)

df.head(50)

,Date,Open,High,Low,Close,Volume,Adj Close,avg_close_5,avg_close_30,std_close_365,avg_vol_5
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,NaN,NaN,NaN,NaN
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,NaN,NaN,NaN,NaN
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,NaN,NaN,NaN,NaN
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,NaN,NaN,NaN,NaN
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,NaN,NaN,NaN,NaN
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900000,NaN,NaN,2046000.0
16583,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974000,NaN,NaN,2226000.0
16582,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022000,NaN,NaN,2374000.0
16581,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.988000,NaN,NaN,2458000.0
16580,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.926000,NaN,NaN,2722000.0


### Dropping Rows with `NaN` and Splitting Dataset into Train and Test

In [4]:
df = df[df["Date"] > datetime(year=1951, month=1, day=2)].dropna(axis=0)
df.isnull().sum()

Date             0
Open             0
High             0
Low              0
Close            0
Volume           0
Adj Close        0
avg_close_5      0
avg_close_30     0
std_close_365    0
avg_vol_5        0
dtype: int64

In [5]:
# The training set is composed of rows before 2013-01-01
train = df[df["Date"] < datetime(year=2013, month=1, day=1)]

# The test set is composed of rows after (including) 2013-01-01
test = df[df["Date"] >= datetime(year=2013, month=1, day=1)]

print(train.shape)
print(test.shape)

(15486, 11)
(739, 11)


### Training the Model

We will use the Linear Regression algorithm and assess its performance by using the Mean Absolute Error (MAE).

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

model = LinearRegression()
features = ['avg_close_5', 'avg_vol_5', 'avg_close_30']
model.fit(train[features], train['Close'])
predictions = model.predict(test[features])
mae = mean_absolute_error(test['Close'], predictions)
print('MAE: ', mae)

MAE:  16.150885859834414


### Conclusion

Using as features `avg_close_5`, `avg_vol_5` and `avg_close_30`, the model displays a `MAE` of 16.2.